## Import the required libraries

In [2]:
import cv2
import numpy as np
from scipy.spatial import distance

## Create object for Background Subtraction and Kernels for erode & dilate

In [3]:
fg_bg = cv2.createBackgroundSubtractorMOG2()
kernel_erode = np.ones((2,2), np.uint8)
kernel_dilate = np.ones((4,4), np.uint8)

## Processing Steps

In [89]:
cap = cv2.VideoCapture('Ant Video.mp4')

while (cap.isOpened()):
    ret, colour_frame = cap.read()
    
    if not ret:
        break
    
    row, col, chn = colour_frame.shape
    
    gray = cv2.cvtColor(colour_frame, cv2.COLOR_BGR2GRAY)
    frame = fg_bg.apply(gray)
    erosion = cv2.erode(frame, kernel_erode, iterations = 2)
    dilation = cv2.dilate(erosion, kernel_dilate, iterations = 4)
    
    ret, frame = cv2.threshold(dilation, 127, 255, 0)
    
    image, contours, hierarchy = cv2.findContours(frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        M = cv2.moments(c)
        cX = int(M['m10'] / M['m00'])
        cY = int(M['m01'] / M['m00']) 
        
        #cv2.drawContours(colour_frame, [c], -1, (255, 0, 255), 2)
        colorim = cv2.circle(colour_frame, (cX, cY), 18, (255, 255, 255), 2)
        cv2.line(colorim, (int(col/2), int(row/2)), (cX, cY), (0,255,0), 2)
        
        dist = distance.euclidean((int(col/2), int(row/2)), (cX, cY))
        colorim = cv2.putText(colorim, str(dist), (cX - 20, cY - 20), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 25, 25),2)
        colorim = cv2.putText(colorim, f'{cX}, {cY}', (cX + 30, cY + 5), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 155),2)
        
    cv2.rectangle(colorim, (250,20), (1000, 90), (0,0,0), 2)
    colorim=cv2.putText(colorim, 'No. of Ants in the Frame : '+ str(len(contours)), (270, 70),
            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2)
    colorim = cv2.circle(colorim, (int(col/2), int(row/2)), 18, (255, 255, 255), -1)
    cv2.imshow('Window', colorim)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()